In [43]:
#### V3

import numpy as np
import csv
import math as m
import scipy.constants as sp

test=0
# constants
k_b=sp.k # boltzman const(k_b) in SI
sig = sp.sigma # stefan boltzmann constant
c=sp.c # speed of light
h=sp.h # Planck's const
pi=np.pi # pi


# emmisivity, refl, trans  csv data transferred into matrix 
e=np.array(list(csv.reader(open("repository/repo/TER_file_cryorun7_emiss.csv","r"),delimiter=','))).astype('float')
d_r=np.array(list(csv.reader(open("repository/repo/TER_file_cryorun7_refl.csv","r"),delimiter=','))).astype('float')
t=np.array(list(csv.reader(open("repository/repo/TER_file_cryorun7_trans.csv","r"),delimiter=','))).astype('float')
s_r=np.array(list(csv.reader(open("repository/repo/spec refl.csv","r"),delimiter=','))).astype('float')

A=[]
A=np.random.rand(6000,13)
AA= A*(10**-21)

In [67]:
n_e = 13
n_k = 6000
view_factor = 1
step_size = 100
t_0 = 300
t_n = 4.2

value_low = 10.**-4
value_high = 1.-value_low

k = 0
Q_total = 0
Q = []
E = []
k_values = []
En=[]
E0= 0
E1= 0
E2= 0
E3= 0
E4= 0
E5= 0
E6= 0
E7= 0
E8= 0
E9= 0
E10= 0
E11= 0
E12= 0

E_mat=[]
for a in range(0, n_k):
    for b in range(0, n_e):
        while (t[a,b] == 0):
            t[a,b]= value_low
        while (d_r[a,b] == 0):
            d_r[a,b]= value_low
        while (s_r[a,b] == 0):
            s_r[a,b]= value_low
        while (e[a,b] == 0):
            e[a,b]= value_low
        

for i in range(0, n_k):
    
    #Do my r_s, r_d, t, e values add up to one (or close to)?
    
    #Resistance
    R_T = 0.
    R_T += d_r[i, 0]/((1-s_r[i, 0])*e[i, 0]) + 1/((1-s_r[i, 0])*(1-s_r[i, 1])*view_factor)
    
    for j in range(1, n_e-1):
        for b in range(0, n_e):
            while (t[a,b] == 1):
                t[a,b]= 1-value_low
            while (d_r[a,b] == 1):
                d_r[a,b]= 1-value_low
            while (s_r[a,b] == 1):
                s_r[a,b]= 1-value_low
            while (e[a,b] == 1):
                e[a,b]= 1-value_low
        R_T += (1/((2*d_r[i, j]/((1-s_r[i, j])*e[i, j]))) + t[i, j]/(1-t[i, j]))**-1 +\
                1/((1-s_r[i, j])*(1-s_r[i, j+1])*view_factor)
            
    R_T += 1/((1-s_r[i, j])*(1-s_r[i, j+1])*view_factor) +\
            d_r[i, j+1]/(1-s_r[i, j+1])*e[i, j+1]
        
    
    k += 100
    
    #Large value correction
    def volt(k, T):
        import decimal as dc
        index = h*c*k/k_b/T
        if (index) > 10**2.8:
            return (0.)
        elif (index) < 10.**(-4):
            return (2*(k**2)*k_b*T)
        else:
            return (2*h*c*k**3)*(1/((m.e**(h*c*k/k_b/T))-1))

    #Current    
   

    I = (volt(k, t_0) - volt(k, t_n))/R_T
    #Q.append(I)
    #Q_total += I*c*step_size*m.pi
    
    #Emissive powers
    E_elements = []
    voltage = volt(k, t_0)
    E_elements.append(voltage)
    
    #first element
    voltage -= I*(d_r[i,0]/(1-s_r[i,0])/e[i,0] + 1/(1-s_r[i,0])/(1-s_r[i,1])/view_factor)
    voltage -= I*((1-t[i,1])/t[i,1])*(d_r[i,1]/(1-s_r[i,1])/e[i,1])/\
               (2*d_r[i,1]/(1-s_r[i,1])/e[i,1] + (1-t[i,1])/t[i,1])
    E_elements.append(voltage)
    
    if test:
        k=1.
        T=1000.
        print(c)
        print(h)
        print(k_b)
        print(volt(k,T))
        print(h*c*k/k_b/T)
        print(2*k**2*k_b*T)
        print(2*h*c*k**3/(m.e**(h*c*k/k_b/T)-1)) 
        a = input('Press enter to continue')
    
    #2nd to last-second element
    for l in range(1, n_e-2):
        voltage -= I*((1-t[i,l])/t[i,l])/(2*d_r[i,l]/(1-s_r[i,l])/e[i,l] +
                                          (1-t[i,l])/t[i,l])*(d_r[i,l]/(1-s_r[i,l])/e[i,l])
        voltage -= I*(1/(1-s_r[i,l])/(1-s_r[i,l+1])/view_factor)
        voltage -= I*((1-t[i,l+1])/t[i,l+1])/(2*d_r[i,l+1]/(1-s_r[i,l+1])/e[i,l+1] + 
                                              (1-t[i,l+1])/t[i,l+1])*(d_r[i,l+1]/(1-s_r[i,l+1])/e[i,l+1])
        E_elements.append(voltage)
  
    #last element
    voltage -= I*((1-t[i,l+1])/t[i,l+1])/(2*d_r[i,l+1]/(1-s_r[i,l+1])/e[i,l+1] +
                                          (1-t[i,l+1])/t[i,l+1])*(d_r[i,l+1]/(1-s_r[i,l+1])/e[i,l+1])
    voltage -= I*(1/(1-s_r[i,l+1])/(1-s_r[i,l+2])/view_factor + 
                  d_r[i,l+2]/(1-s_r[i,l+2])/e[i,l+2])
    E_elements.append(voltage)
    
    E.append(E_elements)
    
    E0= E_elements[0] +E0
    E1= E_elements[1] +E1
    E2= E_elements[2] +E2
    E3= E_elements[3] +E3
    E4= E_elements[4] +E4
    E5= E_elements[5] +E5
    E6= E_elements[6] +E6
    E7= E_elements[7] +E7
    E8= E_elements[8] +E8
    E9= E_elements[9] +E9
    E10= E_elements[10] +E10
    E11= E_elements[11] +E11
    E12= E_elements[12] +E12
    E_elements = []
    A = np.array(E)
    k_values.append(k)
    

A=A#+AA

print (len(E))
E_mat= E[3]
print (E_mat)

6000
[1.3127502432324363e-15, 1.2141344718268708e-15, 1.1155285454644938e-15, 1.0144018270220354e-15, 9.1305757971614234e-16, 8.1335015958423211e-16, 7.1121829313544941e-16, 6.0824021242305092e-16, 5.0617563775084127e-16, 4.0598689069724219e-16, 3.07516668315987e-16, 2.0731797679461343e-16, 1.0711938469782575e-16]


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:62: RuntimeWarning: divide by zero encountered in double_scalars


In [68]:
#print (E0)
#Tn= (En/sig)**(1/4)
EEE = [E0,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12]
Tn=0.
Tn_matrx=[]
#print (EEE)
for iii in range(0, 12):
    Tn = (EEE[iii]/sig)**(1/4)
    Tn_matrx.append(Tn)
print(Tn_matrx)

#print (E)

E_mat= E[3]
print (E_mat)


[0.541537584457286, 0.53914799630777088, 0.52946317030034007, 0.5098519016041505, 0.49689714999733103, 0.47715867548778784, 0.44524774216754154, 0.40159245964947421, 0.34387714890701709, 0.31878569130132778, 0.30645127106550457, 0.26806387535591752]
[1.3127502432324363e-15, 1.2141344718268708e-15, 1.1155285454644938e-15, 1.0144018270220354e-15, 9.1305757971614234e-16, 8.1335015958423211e-16, 7.1121829313544941e-16, 6.0824021242305092e-16, 5.0617563775084127e-16, 4.0598689069724219e-16, 3.07516668315987e-16, 2.0731797679461343e-16, 1.0711938469782575e-16]


In [60]:
while J_in!=J_out
    EEE = (Tn_matrix**4)*sig
    print EEE


SyntaxError: invalid syntax (<ipython-input-60-6a3ea2f7d1f1>, line 1)

In [55]:
EEE = (np.power(Tn_matrx,4))*sig
print(EEE) 

[  4.87670002e-09   4.79119239e-09   4.45609715e-09   3.83166738e-09
   3.45682705e-09   2.93942867e-09   2.22853024e-09   1.47487029e-09
   7.92911830e-10   5.85607874e-10   5.00100554e-10   2.92795768e-10]
